In [49]:
import spacy as sp 
import sklearn as sk 
import numpy as np 
import pandas as pd 
from unicodedata import normalize

nlp = sp.load('de_core_news_sm')

from spacy.lang.de.stop_words import STOP_WORDS



In [50]:
def preprocessTaste(reviewArray):
    postNLP = []
    for review in reviewArray:
        doc = nlp(normalize("NFKD", review))
        filtered =[] 
        for word in doc:
            if word.is_stop == False and not word.is_punct and not word.is_sent_start:
                filtered.append(word.lemma_) 
        postNLP.append(filtered)
    return postNLP

def tasteLemmas(review):
    doc = nlp(review)
    filtered =[] 
    for word in doc:
        if word.is_stop == False and not word.is_punct and not word.is_sent_start:
            filtered.append(word.lemma_) 
    return filtered
    
def identityTokenizer(text):
    return text

from sklearn.feature_extraction.text import CountVectorizer
def vectorizeTaste(reviewArray):
    vectorizer = CountVectorizer(tokenizer = identityTokenizer, lowercase = False)
    vector = vectorizer.fit_transform(preprocessTaste(reviewArray))
    print("Vocab: ", vectorizer.vocabulary_.keys())
    return vector.toarray()

In [55]:
df = pd.read_excel('/home/p/wines/winerecommender/Scraping/Data Sources - scraped/Wines-FinalDS.xlsx')
test = df['Verkostungsnotiz:'].fillna("", inplace=False).to_numpy()
res2 = vectorizeTaste(test)



Vocab:  {'Lisa': 1845, 'Perrotti-Brown': 2275, 'Wine': 3332, 'Advocate': 293, 'Juni': 1616, '2020': 130, 'Opaque': 2187, 'garnet-purple': 5156, 'color': 4219, 'the': 7621, 'Sansonnet': 2630, 'comes': 4226, 'skipping': 7250, 'out': 6421, 'of': 6360, 'glass': 5277, 'with': 8215, 'vibrant': 8015, 'notes': 6322, 'Morello': 2048, 'cherries': 4166, 'blueberry': 3965, 'compote': 4249, 'mulberries': 6223, 'and': 3549, 'warm': 8105, 'black': 3931, 'plums': 6589, 'plus': 6590, 'hints': 5516, 'raspberry': 6779, 'leaves': 5882, 'Sichuan': 2741, 'pepper': 6503, 'cassia': 4108, 'cumin': 4340, 'seed': 7129, 'waft': 8093, 'tree': 7734, 'bark': 3776, 'full-bodied': 5123, 'palate': 6457, 'is': 5677, 'packed': 6448, 'crunchy': 4333, 'red': 6816, 'fruits': 5117, 'framed': 5066, 'by': 4067, 'exquisitely': 4848, 'ripe': 6936, 'rounded': 6970, 'tannins': 7568, 'bold': 3976, 'freshness': 5077, 'finishing': 4991, 'very': 8011, 'long': 5987, 'mineral': 6132, 'laced': 5828, 'Bordeaux': 610, 'Extra': 1030, 'Prime

In [24]:
import json


#load wine data
with open('/home/p/wines/winerecommender/Script - Data Extraction/wine_data.json') as f:
    wines = json.load(f)



#seperate by type
wines = [wine for wine in wines if wine["wine_tastes"] is not None]
white_wines = [wine for wine in wines if wine["wine_type"]=="white"]
red_wines = [wine for wine in wines if wine["wine_type"]=="red"]


In [58]:
#build overall (and potentially group-specific) vocabularies
seen = set()
vocab = []
# groups = ['black_fruit', 'citrus_fruit', 'dried_fruit', 'earth', 'floral', 'microbio', 'non_oak', 'oak', 'red_fruit', 'spices', 'tree_fruit', 'tropical_fruit', 'vegetal']
# group_vocab = {}
# for group in groups:
#     group_vocab |= {group: []}

for wine in wines:
    for group in wine["wine_tastes"]:
        for taste in group["keywords"]:
            if taste[0] not in seen:
                seen.add(taste[0])
                vocab.append(taste[0])
            # if taste[0] not in group_vocab[group["group"]]:
            #     group_vocab[group["group"]].append(taste[0].lower())
vocab = sorted(vocab)
print(vocab)


['5-Gewürzpulver', 'Ahornsirup', 'Akaifrucht', 'Akazie', 'Aleppopfeffer', 'Ananas', 'Angeschlagener Apfel', 'Anis', 'Anissamen', 'Apfel', 'Apfelblüte', 'Aprikose', 'Aprikosenkompott', 'Aprikosenkonfitüre', 'Asche', 'Backgewürz', 'Backpflaume', 'Balsamico', 'Banane', 'Baumrinde', 'Beerenkonfitüre', 'Beerensauce', 'Bergamotte', 'Bickbeere', 'Bienenwachs', 'Birne', 'Bittermandel', 'Bleistiftmine', 'Bleistiftspäne', 'Blockschokolade', 'Blumenerde', 'Blutorange', 'Bohnenkraut', 'Boysenbeere', 'Bratapfel', 'Braten', 'Braune Butter', 'Brauner Zucker', 'Brioche', 'Brombeere', 'Brombeerkonfitüre', 'Brombeersauce', 'Brothefe', 'Butter', 'Cantaloupe-Melone', 'Cassis', 'Castelvetrano-Olive', 'Champignon', 'Cherry Cola', 'Chili', 'Chinin', 'Chutney', 'Cola', 'Cranberry', 'Creme', 'Creme brulee', 'Creme fraiche', 'Curry-Gewürz', 'Dattelpflaume', 'Dill', 'Dosenpfirsich', 'Drachenfrucht', 'Dunkle Frucht', 'Dunkle Schokolade', 'Earl Grey-Tee', 'Echte Limette', 'Eiche', 'Eisen', 'Erbsenschote', 'Erdbeer

In [59]:
#stemming experiments

from nltk.stem import *
from nltk.stem.cistem import *

stemmer = Cistem()

stemmed = [stemmer.stem(word) for word in vocab]

print(stemmed)


['5-gewurzpulv', 'ahornsirup', 'akaifrucht', 'akazie', 'aleppopfeff', 'anana', 'angeschlagener apfel', 'ani', 'anissam', 'apfel', 'apfelblut', 'apriko', 'aprikosenkompott', 'aprikosenkonfitur', 'asche', 'backgewurz', 'backpflaum', 'balsamico', 'bana', 'baumri', 'beerenkonfitur', 'beerensauc', 'bergamott', 'bickbeer', 'bienenwach', 'bir', 'bittermandel', 'bleistiftmi', 'bleistiftspa', 'blockschokolad', 'blumenerd', 'blutorang', 'bohnenkraut', 'boysenbeer', 'bratapfel', 'brat', 'braune butt', 'brauner zuck', 'brioch', 'brombeer', 'brombeerkonfitur', 'brombeersauc', 'brothef', 'butt', 'cantaloupe-melo', 'cassi', 'castelvetrano-oliv', 'champigno', 'cherry cola', 'chili', 'chini', 'chutney', 'cola', 'cranberry', 'crem', 'creme brulee', 'creme fraich', 'curry-gewurz', 'dattelpflaum', 'dill', 'dosenpfirsich', 'drachenfrucht', 'dunkle frucht', 'dunkle schokolad', 'earl grey-tee', 'echte limett', 'eich', 'eise', 'erbsenschot', 'erdbeer', 'erdbeersauc', 'erdig', 'erdnuskrokant', 'erdnuss', 'erdn

In [77]:
#grab subsection of reviews for testing
teststrings = preprocessTaste(df['Verkostungsnotiz:'].fillna("", inplace=False).to_numpy()[0:9])

In [79]:
import re



def scan(vocab, string):
    res = {}
    string = (" ".join(string))
    for word in vocab:
        out = re.findall(word, string)
        res |= {word: len(out)}

    res = {k: v for k, v in res.items() if v > 0}

    return res

for string in teststrings:
    print(string)
    print(scan(vocab, string))



['Lisa', 'Perrotti-Brown', 'Wine', 'Advocate', 'Juni', '2020', 'Opaque', 'garnet-purple', 'color', 'the', 'Sansonnet', 'comes', 'skipping', 'out', 'of', 'the', 'glass', 'with', 'vibrant', 'notes', 'of', 'Morello', 'cherries', 'blueberry', 'compote', 'mulberries', 'and', 'warm', 'black', 'plums', 'plus', 'hints', 'of', 'raspberry', 'leaves', 'Sichuan', 'pepper', 'cassia', 'and', 'cumin', 'seed', 'with', 'waft', 'of', 'tree', 'bark', 'full-bodied', 'palate', 'is', 'packed', 'with', 'crunchy', 'black', 'and', 'red', 'fruits', 'framed', 'by', 'exquisitely', 'ripe', 'rounded', 'tannins', 'and', 'bold', 'freshness', 'finishing', 'very', 'long', 'and', 'mineral', 'laced', 'Bordeaux', 'Extra', 'Primeur', '2019', 'September', '2020', 'Superbe', 'umwerfen', 'fruchtig', 'Aromatik', 'Brombeere', 'Kirsche', 'Ansatz', 'dichter', 'fleischig', 'Bau', 'einmalig', 'feinkörniges', 'saftig', 'Tannin', 'Finale', 'einmalig', 'Präzision', 'zeitgemässe', 'gelungen', 'Interpretation', 'Terroirs', 'Moser', '

In [89]:
from fuzzywuzzy import fuzz, process

s1 = 'Mineralien'
s2 = 'Mineralik'
fuzz.partial_ratio(s1,s2)

print("Partial ratio match for "+s1+" & "+s2+": "+str(fuzz.partial_ratio(s1,s2)))

Partial ratio match for Mineralien & Mineralik: 89
